In [19]:
from sklearn.ensemble import RandomForestRegressor
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from missing_timestamps import remove_duplicates
from data_preparation import Preparation
from gluonts.time_feature import time_features_from_frequency_str, get_lags_for_frequency, get_seasonality



In [20]:
data = Preparation(r'/home/iva/Desktop/operato-meteo-1/data/MAS_processed/ELES-MAS-5001.csv.gz', "AMBIENT_TEMPERATURE")
train, test = data.split()
train_processed = remove_duplicates(train)
#train_processed_ = train_processed["AMBIENT_TEMPERATURE"].copy().squeeze()
train_processed_ = train_processed.asfreq("5T").fillna(method="ffill")
test_processed = remove_duplicates(test)
test_processed_ = test_processed["AMBIENT_TEMPERATURE"].copy().squeeze()

root       INFO  Split the dataset to train and test data.


In [32]:
#train_processed_ = train_processed.asfreq("5T")
papa = train_processed_.isna()
papa[papa["AMBIENT_TEMPERATURE"] == True]

,AMBIENT_TEMPERATURE
timestamp,


In [53]:
train_processed_[train_processed_.index == '2020-10-13 10:55:00']

,AMBIENT_TEMPERATURE
timestamp,
2020-10-13 10:55:00,8.0


In [4]:
lags_from_frequency = get_lags_for_frequency("5min")
seasonality = get_seasonality("5min")


In [25]:
import pandas as pd
train_skforecast = pd.DataFrame()
train_skforecast["y"] = train_processed_["AMBIENT_TEMPERATURE"].values
train_skforecast["datetime"] = train_processed_.index
train_skforecast = train_skforecast.set_index("datetime")

In [28]:
papa = train_skforecast.isna()
idx = papa[papa["y"] == True]
print(idx)

Empty DataFrame
Columns: [y]
Index: []


In [22]:
import numpy as np


In [23]:
train_processed_["AMBIENT_TEMPERATURE"].values
x = np.isnan(train_processed_["AMBIENT_TEMPERATURE"].values
)
x

array([False, False, False, ..., False, False, False])

In [10]:
type(train_processed_["AMBIENT_TEMPERATURE"].values)

numpy.ndarray

In [31]:
forecaster = ForecasterAutoreg(regressor=RandomForestRegressor(random_state=963), lags=lags_from_frequency, differentiation=seasonality)
forecaster.fit(y=train_skforecast["y"])

/home/iva/anaconda3/envs/Python_3.9/lib/python3.9/site-packages/skforecast/utils/utils.py:788: UserWarning: `y` has DatetimeIndex index but no frequency. Index is overwritten with a RangeIndex of step 1.
  warnings.warn(
/home/iva/anaconda3/envs/Python_3.9/lib/python3.9/site-packages/pandas/core/generic.py:2070: RuntimeWarning: overflow encountered in cast
  return np.asarray(self._values, dtype=dtype)
/home/iva/anaconda3/envs/Python_3.9/lib/python3.9/site-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 forecaster = ForecasterAutoreg(regressor=RandomForestRegressor(random_state=963), lags=l     │
│ ❱ 2 forecaster.fit(y=train_skforecast["y"])                                                      │
│   3                                                                                              │
│                                                                                                  │
│ /home/iva/anaconda3/envs/Python_3.9/lib/python3.9/site-packages/skforecast/ForecasterAutoreg/For │
│ ecasterAutoreg.py:534 in fit                                                                     │
│                                                                                                  │
│    531 │   │   │   self.regressor.fit(X=X_train, y=y_train, sample_weight=sample_weight,         │
│    532 │   │   │   │   │   │   │      **self.fit_kwargs)                                         │
│    533 │   │   else:                                                                             │
│ ❱  534 │   │   │   self.regressor.fit(X=X_train, y=y_train, **self.fit_kwargs)                   │
│    535 │   │                                                                                     │
│    536 │   │   self.fitted = True                                                                │
│    537 │   │   self.fit_date = pd.Timestamp.today().strftime('%Y-%m-%d %H:%M:%S')                │
│                                                                                                  │
│ /home/iva/anaconda3/envs/Python_3.9/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:331  │
│ in fit                                                                                           │
│                                                                                                  │
│    328 │   │   # Validate or convert input data                                                  │
│    329 │   │   if issparse(y):                                                                   │
│    330 │   │   │   raise ValueError("sparse multilabel-indicator for y is not supported.")       │
│ ❱  331 │   │   X, y = self._validate_data(                                                       │
│    332 │   │   │   X, y, multi_output=True, accept_sparse="csc", dtype=DTYPE                     │
│    333 │   │   )                                                                                 │
│    334 │   │   if sample_weight is not None:                                                     │
│                                                                                                  │
│ /home/iva/anaconda3/envs/Python_3.9/lib/python3.9/site-packages/sklearn/base.py:596 in           │
│ _validate_data                                                                                   │
│                                                                                                  │
│    593 │   │   │   │   │   check_y_params = {**default_check_params, **check_y_params}           │
│    594 │   │   │   │   y = check_array(y, input_name="y", **check_y_params)                      │
│    595 │   │   │   else:                                                                         │
│ ❱  596 │   │   │   │   X, y = check_X_y(X, y, **check_params)                                    │
│    597 │   │   │   out = X, y                                                                    │
│    598 │   │                                                                                     │
│    599 │   │   if not no_val_X and check_params.get("ensure_2d", True):                          │
│                                                            

In [7]:
import pandas as pd
# Download data
# ==============================================================================
url = (
    'https://raw.githubusercontent.com/JoaquinAmatRodrigo/skforecast/master/'
    'data/h2o.csv'
)
data = pd.read_csv(url, sep=',', header=0, names=['y', 'datetime'])

# Data preprocessing
# ==============================================================================
data['datetime'] = pd.to_datetime(data['datetime'], format='%Y-%m-%d')
data = data.set_index('datetime')
data = data.asfreq('MS')
#data = data['y']
#data = data.sort_index()
data

,y
datetime,
1991-07-01,0.429795
1991-08-01,0.400906
1991-09-01,0.432159
1991-10-01,0.492543
1991-11-01,0.502369
...,...
2008-02-01,0.761822
2008-03-01,0.649435
2008-04-01,0.827887


In [12]:
data["y"].values

array([0.429795  , 0.400906  , 0.432159  , 0.492543  , 0.502369  ,
       0.602652  , 0.660119  , 0.33622   , 0.351348  , 0.379808  ,
       0.361801  , 0.410534  , 0.48338867, 0.47546342, 0.53476104,
       0.56860613, 0.59522329, 0.77125778, 0.7515028 , 0.38755434,
       0.42728322, 0.41389018, 0.42885882, 0.47012642, 0.50920969,
       0.558443  , 0.60151406, 0.6329471 , 0.69960539, 0.96308051,
       0.81932534, 0.4376698 , 0.50612127, 0.47049117, 0.51069626,
       0.54051379, 0.55811892, 0.67285206, 0.68589738, 0.68969198,
       0.74130358, 0.81330763, 0.80311257, 0.47525824, 0.5525723 ,
       0.52710782, 0.56124982, 0.58897764, 0.62313362, 0.74083723,
       0.72537176, 0.81580302, 0.81400947, 0.92665305, 0.93727594,
       0.52876165, 0.55933994, 0.57787166, 0.61492741, 0.59418877,
       0.70775844, 0.71950195, 0.74432369, 0.80485511, 0.78854235,
       0.9710894 , 0.84683354, 0.46382252, 0.48527317, 0.5280586 ,
       0.56233647, 0.5885704 , 0.66948036, 0.67799365, 0.76299